In [13]:
import os
import torch
import pandas as pd
from datasets import NoduleDataset, SubtypedDataLoader

# Testing SubtypedDataLoader

In [3]:
lidc_subtyped = pd.read_csv('lidc_subtyped.csv')
lidc_subtyped.head(5)

,Unnamed: 0,Nodule_id,malignancy,subtype
0,0,1197,benign,0benign
1,1,1208,benign,1benign
2,2,1109,benign,1benign
3,3,1259,benign,0benign
4,4,1050,benign,1benign


In [8]:
def getNormed(this_array, this_min = 0, this_max = 255, set_to_int = True):
    
    rat = (this_max - this_min)/(this_array.max() - this_array.min())
    this_array = this_array * rat
    this_array -= this_array.min()
    this_array += this_min
    if set_to_int:
        return this_array.to(dtype= torch.int)
    return this_array

In [14]:
def getImages(image_folder):
    '''
        Input:
        image_folder: directory of the image files

        Output:
        m1: list of the labels encountered (1,2,4,5)
        m2: list of binary labels encountered (benign, malignant)
        diff: list of any nodes with discrepency to CSV labels

    '''
    
    M_benign = []
    L_benign = []
    
    M_malignant = []
    L_malignant = []

    

    lidc = pd.read_csv('lidc_subtyped.csv')
    for dir1 in os.listdir(image_folder):
  
        if dir1 == 'Malignancy_3':
            continue

        for file in os.listdir(os.path.join(image_folder, dir1)):
            malignancy_orig = int(dir1[-1])
            m1.append(malignancy_orig)

            temp_nodule_ID = file.split('.')[0]
            subtype = lidc[lidc['noduleID']==int(temp_nodule_ID)]['subtype'].iloc[0] 
            
            if subtype == '0benign':
                image_array = M_benign
            elif subtype == '1benign':
                image_array = L_benign
            elif subtype == '1malignant':
                image_array = M_malignant
            else:
                image_array = L_malignant
            
            image = np.loadtxt(os.path.join(image_folder, dir1,file))
            image = torch.from_numpy(image).to(DEVICE)
            rgb_image = torch.stack((image,image,image), dim = 0)
            rgb_image = getNormed(rgb_image)
            rgb_image = rgb_image / 255 

            image_array.append(rgb_image)


    return M_benign, L_benign, M_malignant, L_malignant


In [15]:
M_benign, L_benign, M_malignant, L_malignant = getImages('LIDC(MaxSlices)_Nodules')

ValueError: invalid literal for int() with base 10: 'd'